In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [3]:
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [4]:
from importlib import reload  
utils = reload(utils)

2.5.0
keras
2.5.0


In [5]:
dataset = 'intel'

In [6]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [7]:
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/MNIST project/models2"

    
model = utils.My_model(dataset,True, model_dir)

Model loaded correctly


In [8]:
model.evaluate(x_test,y_test)

94/94 [==============================] - 40s 40ms/step - loss: 0.5392 - accuracy: 0.8037


[0.5391883850097656, 0.8036666512489319]

In [9]:
y_test_labels = np.argmax(y_test,axis=1)
print(y_test_labels[:10])

[0 0 0 0 0 0 0 0 0 0]


In [10]:
y_train_labels = np.argmax(y_train,axis=1)
print(y_train_labels[:10])

[0 0 0 0 0 0 0 0 0 0]


In [11]:
x_train.shape[1:3]

(48, 48)

In [12]:
img_width,img_height  = model.sizes
print(img_width,img_height)

48 48


In [14]:
loading_cw = True
loading_bim = True
loading_fgsm = True

In [17]:
n_advs = 3000

In [18]:
see_images = False

## Adversarial methods

In [19]:
def see_x_image(x,y):
    '''# (48,48,1) show image, y - (43,)
    '''
    plt.figure()
    
    plt.imshow((x.reshape((img_width,img_height))*255).astype("uint8"))
    plt.title(str(np.argmax(y)))
    plt.axis("off")


In [20]:
import foolbox as fb
import keras.backend as K

In [21]:
epsilons =[0.01,0.1]
epsilons =[0.05]

In [22]:
#pip install foolbox --upgrade

In [23]:
from keras.utils import np_utils

### Creating adversarial set from training set

In [21]:
fmodel = fb.TensorFlowModel(model.model,bounds=(0,1))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [22]:

n_each_atack = int(n_advs/3)
print(n_each_atack)

1000


In [23]:
# Create n random numbers to get inputs from a set
n = n_advs
different_idxs_train = np.random.choice(len(x_train), n, replace=False)
print(len(different_idxs_train))
print(different_idxs_train[:10])

3000
[ 1469 10129 10856  4727  5546   480  1968  7797  5511   792]


In [24]:
# Create n random numbers to get inputs from a set
n = n_advs
different_idxs_test = np.random.choice(len(x_test), n, replace=False)
print(len(different_idxs_test))
print(different_idxs_test[:10])

3000
[2379  423 1590 2624 1970 2618  302 2987 2393 1519]


In [25]:
type(different_idxs_train)

numpy.ndarray

In [26]:
idxs_cw_train = different_idxs_train[:n_each_atack]
idxs_bim_train = different_idxs_train[n_each_atack:2*n_each_atack]
idxs_fgsm_train = different_idxs_train[2*n_each_atack:]
print(len(idxs_cw_train))
print(len(idxs_bim_train))
print(len(idxs_fgsm_train))

1000
1000
1000


In [27]:
idxs_cw_test = different_idxs_test[:n_each_atack]
idxs_bim_test = different_idxs_test[n_each_atack:2*n_each_atack]
idxs_fgsm_test = different_idxs_test[2*n_each_atack:]
print(len(idxs_cw_test))
print(len(idxs_bim_test))
print(len(idxs_fgsm_test))

1000
1000
1000


In [28]:
print(idxs_cw_train[:10])
print(idxs_cw_test[:10])

[ 1469 10129 10856  4727  5546   480  1968  7797  5511   792]
[2379  423 1590 2624 1970 2618  302 2987 2393 1519]


In [29]:
#loading = True

In [30]:
if (loading_cw):
    print("Loading...")
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_cw_train.npy"
    idxs_cw_train = np.load(save_dir)
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_bim_train.npy"
    idxs_bim_train = np.load(save_dir)
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_fgsm_train.npy"
    idxs_fgsm_train = np.load(save_dir)

In [24]:
if (loading_cw):
    print("Loading...")
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_cw_test.npy"
    idxs_cw_test = np.load(save_dir)
    print(idxs_cw_test.shape)
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_bim_test.npy"
    idxs_bim_test = np.load(save_dir)
    print(idxs_bim_test.shape)
    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_fgsm_test.npy"
    idxs_fgsm_test = np.load(save_dir)
    print(idxs_fgsm_test.shape)
    

Loading...
(1000,)
(1000,)
(1000,)


In [32]:
idxs_cw_train[:10]

array([ 1469, 10129, 10856,  4727,  5546,   480,  1968,  7797,  5511,
         792])

In [33]:
if (not loading_cw):
    #print("saving")
    #save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_advs.npy"

    #with open(save_dir, 'wb') as f:
    #    np.save(f, different_idxs)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_cw_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_cw_train)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_bim_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_bim_train)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_fgsm_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_fgsm_train)

In [34]:
if (not loading_cw):
    #print("saving")
    #save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_advs.npy"

    #with open(save_dir, 'wb') as f:
    #    np.save(f, different_idxs)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_cw_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_cw_test)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_bim_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_bim_test)

    save_dir = "D:/data_adversarial_july/"+dataset+"/idxs_fgsm_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, idxs_fgsm_test)

In [35]:
def get_from_original_set_adv(idx_list,x_set,y_set_labels):
    new_images = np.zeros((len(idx_list),img_width,img_height,1))
    new_labels = np.zeros((len(idx_list),))
    j=0
    for idx in idx_list:
        print(idx)
        print("---",j)
        new_images[j] = x_set[int(idx)]
        new_labels[j] = y_set_labels[int(idx)]
        print("label",new_labels[j])
        j +=1
    return new_images , new_labels

In [36]:
images_to_adv_train,labels_to_adv_train  = get_from_original_set_adv(idxs_cw_train,x_train,y_train_labels)

1469
--- 0
label 0.0
10129
--- 1
label 5.0
10856
--- 2
label 5.0
4727
--- 3
label 2.0
5546
--- 4
label 3.0
480
--- 5
label 0.0
1968
--- 6
label 1.0
7797
--- 7
label 4.0
5511
--- 8
label 3.0
792
--- 9
label 0.0
5556
--- 10
label 3.0
3619
--- 11
label 2.0
5315
--- 12
label 2.0
964
--- 13
label 0.0
2689
--- 14
label 1.0
3959
--- 15
label 2.0
7241
--- 16
label 3.0
8401
--- 17
label 4.0
9692
--- 18
label 5.0
5962
--- 19
label 3.0
5064
--- 20
label 2.0
5257
--- 21
label 2.0
10303
--- 22
label 5.0
8940
--- 23
label 4.0
8596
--- 24
label 4.0
10831
--- 25
label 5.0
4451
--- 26
label 2.0
1423
--- 27
label 0.0
220
--- 28
label 0.0
5654
--- 29
label 3.0
4248
--- 30
label 2.0
3258
--- 31
label 1.0
9014
--- 32
label 4.0
3482
--- 33
label 1.0
5681
--- 34
label 3.0
4057
--- 35
label 2.0
8280
--- 36
label 4.0
7011
--- 37
label 3.0
5826
--- 38
label 3.0
11223
--- 39
label 5.0
9039
--- 40
label 4.0
522
--- 41
label 0.0
1209
--- 42
label 0.0
3131
--- 43
label 1.0
9003
--- 44
label 4.0
6231
--- 45
label 3.

--- 443
label 1.0
1307
--- 444
label 0.0
5699
--- 445
label 3.0
1168
--- 446
label 0.0
27
--- 447
label 0.0
8992
--- 448
label 4.0
8975
--- 449
label 4.0
2166
--- 450
label 1.0
7256
--- 451
label 3.0
8002
--- 452
label 4.0
4008
--- 453
label 2.0
3542
--- 454
label 1.0
5431
--- 455
label 2.0
2372
--- 456
label 1.0
6170
--- 457
label 3.0
4186
--- 458
label 2.0
2316
--- 459
label 1.0
7625
--- 460
label 4.0
6764
--- 461
label 3.0
4123
--- 462
label 2.0
678
--- 463
label 0.0
2016
--- 464
label 1.0
8655
--- 465
label 4.0
10013
--- 466
label 5.0
2761
--- 467
label 1.0
3985
--- 468
label 2.0
4158
--- 469
label 2.0
1007
--- 470
label 0.0
10456
--- 471
label 5.0
2541
--- 472
label 1.0
7789
--- 473
label 4.0
10927
--- 474
label 5.0
7046
--- 475
label 3.0
2569
--- 476
label 1.0
5949
--- 477
label 3.0
5602
--- 478
label 3.0
5583
--- 479
label 3.0
7281
--- 480
label 3.0
7237
--- 481
label 3.0
914
--- 482
label 0.0
6607
--- 483
label 3.0
9115
--- 484
label 4.0
2957
--- 485
label 1.0
6830
--- 486
labe

label 1.0
5168
--- 896
label 2.0
3869
--- 897
label 2.0
2426
--- 898
label 1.0
9206
--- 899
label 4.0
376
--- 900
label 0.0
3392
--- 901
label 1.0
8493
--- 902
label 4.0
6312
--- 903
label 3.0
8764
--- 904
label 4.0
2432
--- 905
label 1.0
9890
--- 906
label 5.0
7799
--- 907
label 4.0
4015
--- 908
label 2.0
6021
--- 909
label 3.0
640
--- 910
label 0.0
6029
--- 911
label 3.0
3249
--- 912
label 1.0
3606
--- 913
label 2.0
10368
--- 914
label 5.0
3612
--- 915
label 2.0
10501
--- 916
label 5.0
547
--- 917
label 0.0
7966
--- 918
label 4.0
1708
--- 919
label 0.0
3025
--- 920
label 1.0
3199
--- 921
label 1.0
4774
--- 922
label 2.0
4195
--- 923
label 2.0
6498
--- 924
label 3.0
5206
--- 925
label 2.0
9519
--- 926
label 5.0
1868
--- 927
label 1.0
8123
--- 928
label 4.0
9663
--- 929
label 5.0
4517
--- 930
label 2.0
7031
--- 931
label 3.0
8924
--- 932
label 4.0
2277
--- 933
label 1.0
3279
--- 934
label 1.0
7806
--- 935
label 4.0
9617
--- 936
label 5.0
5562
--- 937
label 3.0
290
--- 938
label 0.0
314

In [37]:
images_to_adv_test,labels_to_adv_test  = get_from_original_set_adv(idxs_cw_test,x_test,y_test_labels)

2379
--- 0
label 4.0
423
--- 1
label 0.0
1590
--- 2
label 3.0
2624
--- 3
label 5.0
1970
--- 4
label 3.0
2618
--- 5
label 5.0
302
--- 6
label 0.0
2987
--- 7
label 5.0
2393
--- 8
label 4.0
1519
--- 9
label 3.0
370
--- 10
label 0.0
1846
--- 11
label 3.0
2551
--- 12
label 5.0
874
--- 13
label 1.0
72
--- 14
label 0.0
361
--- 15
label 0.0
2118
--- 16
label 4.0
1243
--- 17
label 2.0
391
--- 18
label 0.0
247
--- 19
label 0.0
661
--- 20
label 1.0
2781
--- 21
label 5.0
1638
--- 22
label 3.0
1382
--- 23
label 2.0
2845
--- 24
label 5.0
173
--- 25
label 0.0
1011
--- 26
label 2.0
2032
--- 27
label 4.0
2798
--- 28
label 5.0
1074
--- 29
label 2.0
1264
--- 30
label 2.0
533
--- 31
label 1.0
1466
--- 32
label 3.0
2511
--- 33
label 5.0
1890
--- 34
label 3.0
908
--- 35
label 1.0
1709
--- 36
label 3.0
1141
--- 37
label 2.0
1384
--- 38
label 2.0
135
--- 39
label 0.0
1991
--- 40
label 4.0
2958
--- 41
label 5.0
586
--- 42
label 1.0
1494
--- 43
label 3.0
2491
--- 44
label 4.0
2091
--- 45
label 4.0
1802
--- 46
l

label 4.0
2408
--- 418
label 4.0
849
--- 419
label 1.0
1854
--- 420
label 3.0
2056
--- 421
label 4.0
2885
--- 422
label 5.0
2089
--- 423
label 4.0
848
--- 424
label 1.0
860
--- 425
label 1.0
1238
--- 426
label 2.0
1644
--- 427
label 3.0
732
--- 428
label 1.0
2969
--- 429
label 5.0
1680
--- 430
label 3.0
420
--- 431
label 0.0
132
--- 432
label 0.0
2807
--- 433
label 5.0
2527
--- 434
label 5.0
0
--- 435
label 0.0
1926
--- 436
label 3.0
327
--- 437
label 0.0
2116
--- 438
label 4.0
20
--- 439
label 0.0
2055
--- 440
label 4.0
2099
--- 441
label 4.0
453
--- 442
label 1.0
793
--- 443
label 1.0
2368
--- 444
label 4.0
1549
--- 445
label 3.0
697
--- 446
label 1.0
1723
--- 447
label 3.0
2717
--- 448
label 5.0
2797
--- 449
label 5.0
1902
--- 450
label 3.0
2350
--- 451
label 4.0
1112
--- 452
label 2.0
2304
--- 453
label 4.0
2236
--- 454
label 4.0
1310
--- 455
label 2.0
853
--- 456
label 1.0
657
--- 457
label 1.0
2297
--- 458
label 4.0
1800
--- 459
label 3.0
1684
--- 460
label 3.0
163
--- 461
label 

label 3.0
2312
--- 791
label 4.0
832
--- 792
label 1.0
1908
--- 793
label 3.0
1154
--- 794
label 2.0
1093
--- 795
label 2.0
654
--- 796
label 1.0
1120
--- 797
label 2.0
114
--- 798
label 0.0
1533
--- 799
label 3.0
46
--- 800
label 0.0
2329
--- 801
label 4.0
1995
--- 802
label 4.0
501
--- 803
label 1.0
576
--- 804
label 1.0
619
--- 805
label 1.0
412
--- 806
label 0.0
2299
--- 807
label 4.0
1561
--- 808
label 3.0
1797
--- 809
label 3.0
1934
--- 810
label 3.0
1999
--- 811
label 4.0
2866
--- 812
label 5.0
1620
--- 813
label 3.0
1062
--- 814
label 2.0
2514
--- 815
label 5.0
300
--- 816
label 0.0
276
--- 817
label 0.0
609
--- 818
label 1.0
2434
--- 819
label 4.0
739
--- 820
label 1.0
2590
--- 821
label 5.0
680
--- 822
label 1.0
2713
--- 823
label 5.0
2248
--- 824
label 4.0
2907
--- 825
label 5.0
823
--- 826
label 1.0
799
--- 827
label 1.0
1342
--- 828
label 2.0
757
--- 829
label 1.0
2975
--- 830
label 5.0
1335
--- 831
label 2.0
1609
--- 832
label 3.0
2360
--- 833
label 4.0
1795
--- 834
label

In [38]:
labels_to_adv_train=tf.convert_to_tensor(labels_to_adv_train)
images_to_adv_train=tf.convert_to_tensor(images_to_adv_train)
images_to_adv_train = tf.cast(images_to_adv_train, tf.float32)
labels_to_adv_train = tf.cast(labels_to_adv_train, tf.int32)

In [39]:
labels_to_adv_test = tf.convert_to_tensor(labels_to_adv_test)
images_to_adv_test = tf.convert_to_tensor(images_to_adv_test)
images_to_adv_test = tf.cast(images_to_adv_test, tf.float32)
labels_to_adv_test = tf.cast(labels_to_adv_test, tf.int32)

In [40]:
attack_CW = fb.attacks.L2BasicIterativeAttack()


In [41]:
start_time = datetime.now()

raw_train, clipped, is_adv = attack_CW(fmodel, images_to_adv_train, labels_to_adv_train,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:26.052493


In [42]:
start_time = datetime.now()

raw_test, clipped, is_adv = attack_CW(fmodel, images_to_adv_test, labels_to_adv_test,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:29.947532


In [43]:
#fb.attacks.p
# new train set
numpy_raw_train = K.eval(raw_train)
numpy_labels_train = K.eval(labels_to_adv_train)

In [44]:
#fb.attacks.p
# new test set
numpy_raw_test = K.eval(raw_test)
numpy_labels_test = K.eval(labels_to_adv_test)

In [45]:
labels_to_adv_onehot_train =np_utils.to_categorical(labels_to_adv_train, model.num_classes)

In [46]:
labels_to_adv_onehot_test =np_utils.to_categorical(labels_to_adv_test, model.num_classes)

In [47]:
if(see_images):
    see_x_image(x_train[33693],y_train[33693])
    see_x_image(numpy_raw_train[0],labels_to_adv_onehot_train[0])
    see_x_image(numpy_raw_train[0]-x_train[33693],labels_to_adv_onehot_train[0])

In [48]:
if(see_images):
    see_x_image(x_test[3699],y_test[3699])
    see_x_image(numpy_raw_test[0],labels_to_adv_onehot_test[0])
    see_x_image(numpy_raw_test[0]-x_test[3699],labels_to_adv_onehot_test[0])

In [49]:
labels_to_adv_onehot_train[0]

array([1., 0., 0., 0., 0., 0.], dtype=float32)

In [50]:
loading_cw

False

In [26]:
if (loading_cw):
    print("loading...")
    numpy_raw_train = np.load("D:/data_adversarial_july/"+dataset+"/images_cw_train.npy")
    labels_to_adv_onehot_train = np.load("D:/data_adversarial_july/"+dataset+"/labels_cw_train.npy")
    
    numpy_raw_test = np.load("D:/data_adversarial_july/"+dataset+"/images_cw_test.npy")
    labels_to_adv_onehot_test = np.load("D:/data_adversarial_july/"+dataset+"/labels_cw_test.npy")
    print("finished")

loading...
finished


In [27]:
model.evaluate(numpy_raw_test,labels_to_adv_onehot_test)

32/32 [==============================] - 1s 40ms/step - loss: 0.6610 - accuracy: 0.7540


[0.6609549522399902, 0.7540000081062317]

In [53]:
if(not loading_cw):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_cw_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_train)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_cw_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_train)

In [54]:
if (not loading_cw):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_cw_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_test)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_cw_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_test)

### BIM

In [55]:
#loading_bim =True

In [56]:
images_to_adv_train,labels_to_adv_train  = get_from_original_set_adv(idxs_bim_train,x_train,y_train_labels)

8967
--- 0
label 4.0
4293
--- 1
label 2.0
10952
--- 2
label 5.0
6637
--- 3
label 3.0
568
--- 4
label 0.0
6755
--- 5
label 3.0
5161
--- 6
label 2.0
2135
--- 7
label 1.0
8839
--- 8
label 4.0
8244
--- 9
label 4.0
4725
--- 10
label 2.0
6040
--- 11
label 3.0
3096
--- 12
label 1.0
1779
--- 13
label 1.0
10621
--- 14
label 5.0
8425
--- 15
label 4.0
5771
--- 16
label 3.0
8705
--- 17
label 4.0
5092
--- 18
label 2.0
11007
--- 19
label 5.0
4220
--- 20
label 2.0
4155
--- 21
label 2.0
6242
--- 22
label 3.0
7580
--- 23
label 4.0
10157
--- 24
label 5.0
9047
--- 25
label 4.0
5213
--- 26
label 2.0
4022
--- 27
label 2.0
9951
--- 28
label 5.0
1549
--- 29
label 0.0
10838
--- 30
label 5.0
6627
--- 31
label 3.0
8817
--- 32
label 4.0
4807
--- 33
label 2.0
1296
--- 34
label 0.0
6570
--- 35
label 3.0
2025
--- 36
label 1.0
108
--- 37
label 0.0
1226
--- 38
label 0.0
6863
--- 39
label 3.0
10409
--- 40
label 5.0
6578
--- 41
label 3.0
104
--- 42
label 0.0
9117
--- 43
label 4.0
9150
--- 44
label 4.0
2485
--- 45
label

label 2.0
853
--- 419
label 0.0
4975
--- 420
label 2.0
5785
--- 421
label 3.0
3980
--- 422
label 2.0
7817
--- 423
label 4.0
3888
--- 424
label 2.0
11104
--- 425
label 5.0
8866
--- 426
label 4.0
4961
--- 427
label 2.0
10286
--- 428
label 5.0
9218
--- 429
label 4.0
10280
--- 430
label 5.0
2927
--- 431
label 1.0
9916
--- 432
label 5.0
10485
--- 433
label 5.0
2556
--- 434
label 1.0
9879
--- 435
label 5.0
6654
--- 436
label 3.0
6959
--- 437
label 3.0
6703
--- 438
label 3.0
4140
--- 439
label 2.0
1672
--- 440
label 0.0
862
--- 441
label 0.0
9561
--- 442
label 5.0
5477
--- 443
label 2.0
6813
--- 444
label 3.0
2644
--- 445
label 1.0
9278
--- 446
label 4.0
4650
--- 447
label 2.0
1406
--- 448
label 0.0
1831
--- 449
label 1.0
4603
--- 450
label 2.0
8389
--- 451
label 4.0
5874
--- 452
label 3.0
6602
--- 453
label 3.0
9027
--- 454
label 4.0
6249
--- 455
label 3.0
1169
--- 456
label 0.0
10179
--- 457
label 5.0
422
--- 458
label 0.0
7388
--- 459
label 3.0
7782
--- 460
label 4.0
2912
--- 461
label 1.0

label 1.0
10195
--- 781
label 5.0
7066
--- 782
label 3.0
6114
--- 783
label 3.0
10154
--- 784
label 5.0
3020
--- 785
label 1.0
10759
--- 786
label 5.0
926
--- 787
label 0.0
8859
--- 788
label 4.0
4869
--- 789
label 2.0
5694
--- 790
label 3.0
5016
--- 791
label 2.0
3838
--- 792
label 2.0
2462
--- 793
label 1.0
5231
--- 794
label 2.0
7641
--- 795
label 4.0
210
--- 796
label 0.0
2249
--- 797
label 1.0
8533
--- 798
label 4.0
4279
--- 799
label 2.0
4407
--- 800
label 2.0
4800
--- 801
label 2.0
4396
--- 802
label 2.0
9607
--- 803
label 5.0
18
--- 804
label 0.0
6056
--- 805
label 3.0
3319
--- 806
label 1.0
2007
--- 807
label 1.0
7510
--- 808
label 4.0
6272
--- 809
label 3.0
9489
--- 810
label 5.0
6294
--- 811
label 3.0
6642
--- 812
label 3.0
8886
--- 813
label 4.0
6131
--- 814
label 3.0
515
--- 815
label 0.0
5494
--- 816
label 3.0
7849
--- 817
label 4.0
1989
--- 818
label 1.0
3160
--- 819
label 1.0
5615
--- 820
label 3.0
410
--- 821
label 0.0
2297
--- 822
label 1.0
1027
--- 823
label 0.0
7874

In [57]:
images_to_adv_test,labels_to_adv_test  = get_from_original_set_adv(idxs_bim_test,x_test,y_test_labels)

1896
--- 0
label 3.0
228
--- 1
label 0.0
719
--- 2
label 1.0
601
--- 3
label 1.0
1755
--- 4
label 3.0
1524
--- 5
label 3.0
769
--- 6
label 1.0
1937
--- 7
label 3.0
2588
--- 8
label 5.0
1287
--- 9
label 2.0
1683
--- 10
label 3.0
2257
--- 11
label 4.0
712
--- 12
label 1.0
2337
--- 13
label 4.0
242
--- 14
label 0.0
1608
--- 15
label 3.0
540
--- 16
label 1.0
1849
--- 17
label 3.0
1306
--- 18
label 2.0
1164
--- 19
label 2.0
985
--- 20
label 2.0
2494
--- 21
label 4.0
2730
--- 22
label 5.0
919
--- 23
label 2.0
545
--- 24
label 1.0
452
--- 25
label 1.0
2178
--- 26
label 4.0
1295
--- 27
label 2.0
1332
--- 28
label 2.0
1149
--- 29
label 2.0
863
--- 30
label 1.0
207
--- 31
label 0.0
2813
--- 32
label 5.0
2686
--- 33
label 5.0
765
--- 34
label 1.0
2274
--- 35
label 4.0
525
--- 36
label 1.0
2752
--- 37
label 5.0
692
--- 38
label 1.0
2834
--- 39
label 5.0
1993
--- 40
label 4.0
2586
--- 41
label 5.0
1701
--- 42
label 3.0
2784
--- 43
label 5.0
2468
--- 44
label 4.0
348
--- 45
label 0.0
616
--- 46
labe

2065
--- 387
label 4.0
927
--- 388
label 2.0
83
--- 389
label 0.0
2155
--- 390
label 4.0
105
--- 391
label 0.0
1078
--- 392
label 2.0
187
--- 393
label 0.0
136
--- 394
label 0.0
2051
--- 395
label 4.0
1962
--- 396
label 3.0
709
--- 397
label 1.0
2544
--- 398
label 5.0
2036
--- 399
label 4.0
2707
--- 400
label 5.0
222
--- 401
label 0.0
2516
--- 402
label 5.0
1986
--- 403
label 3.0
1722
--- 404
label 3.0
322
--- 405
label 0.0
2019
--- 406
label 4.0
2139
--- 407
label 4.0
1829
--- 408
label 3.0
2399
--- 409
label 4.0
2211
--- 410
label 4.0
386
--- 411
label 0.0
1106
--- 412
label 2.0
1803
--- 413
label 3.0
1838
--- 414
label 3.0
1017
--- 415
label 2.0
2632
--- 416
label 5.0
669
--- 417
label 1.0
169
--- 418
label 0.0
328
--- 419
label 0.0
2755
--- 420
label 5.0
2695
--- 421
label 5.0
2504
--- 422
label 5.0
1418
--- 423
label 2.0
2475
--- 424
label 4.0
1065
--- 425
label 2.0
2601
--- 426
label 5.0
1791
--- 427
label 3.0
2174
--- 428
label 4.0
1272
--- 429
label 2.0
1167
--- 430
label 2.0
2

label 2.0
368
--- 827
label 0.0
2801
--- 828
label 5.0
286
--- 829
label 0.0
2247
--- 830
label 4.0
2121
--- 831
label 4.0
2648
--- 832
label 5.0
1566
--- 833
label 3.0
951
--- 834
label 2.0
844
--- 835
label 1.0
1547
--- 836
label 3.0
1541
--- 837
label 3.0
2313
--- 838
label 4.0
1028
--- 839
label 2.0
1481
--- 840
label 3.0
184
--- 841
label 0.0
2191
--- 842
label 4.0
441
--- 843
label 1.0
666
--- 844
label 1.0
1876
--- 845
label 3.0
2122
--- 846
label 4.0
1357
--- 847
label 2.0
393
--- 848
label 0.0
883
--- 849
label 1.0
2554
--- 850
label 5.0
1319
--- 851
label 2.0
36
--- 852
label 0.0
2887
--- 853
label 5.0
2940
--- 854
label 5.0
259
--- 855
label 0.0
1972
--- 856
label 3.0
2006
--- 857
label 4.0
2610
--- 858
label 5.0
2888
--- 859
label 5.0
689
--- 860
label 1.0
2661
--- 861
label 5.0
1532
--- 862
label 3.0
2384
--- 863
label 4.0
2335
--- 864
label 4.0
809
--- 865
label 1.0
1183
--- 866
label 2.0
1614
--- 867
label 3.0
1888
--- 868
label 3.0
1378
--- 869
label 2.0
1832
--- 870
la

In [58]:
labels_to_adv_train=tf.convert_to_tensor(labels_to_adv_train)
images_to_adv_train=tf.convert_to_tensor(images_to_adv_train)
images_to_adv_train = tf.cast(images_to_adv_train, tf.float32)
labels_to_adv_train = tf.cast(labels_to_adv_train, tf.int32)

In [59]:
labels_to_adv_test = tf.convert_to_tensor(labels_to_adv_test)
images_to_adv_test = tf.convert_to_tensor(images_to_adv_test)
images_to_adv_test = tf.cast(images_to_adv_test, tf.float32)
labels_to_adv_test = tf.cast(labels_to_adv_test, tf.int32)

In [60]:
attack_BIM = fb.attacks.L1BasicIterativeAttack()

In [61]:
start_time = datetime.now()

raw_train, clipped, is_adv = attack_BIM(fmodel, images_to_adv_train, labels_to_adv_train,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:35.671826


In [62]:
start_time = datetime.now()

raw_test, clipped, is_adv = attack_BIM(fmodel, images_to_adv_test, labels_to_adv_test,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:43.850614


In [63]:
numpy_raw_train = K.eval(raw_train)
numpy_labels_train = K.eval(labels_to_adv_train)

In [64]:
numpy_raw_test = K.eval(raw_test)
numpy_labels_test = K.eval(labels_to_adv_test)

In [65]:
labels_to_adv_onehot_train =np_utils.to_categorical(labels_to_adv_train, model.num_classes)

In [66]:
labels_to_adv_onehot_test =np_utils.to_categorical(labels_to_adv_test, model.num_classes)

In [67]:
if(see_images):
    see_x_image(x_train[19055],y_train[19055])
    see_x_image(numpy_raw_train[0],labels_to_adv_onehot_train[0])
    see_x_image(numpy_raw_train[0]-x_train[19055],labels_to_adv_onehot_train[0])

In [68]:
if(see_images):
    see_x_image(x_test[6128],y_test[6128])
    see_x_image(numpy_raw_test[0],labels_to_adv_onehot_test[0])
    see_x_image(numpy_raw_test[0]-x_test[6128],labels_to_adv_onehot_test[0])

In [69]:
if(not loading_bim):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_bim_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_train)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_bim_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_train)

In [70]:
if (not loading_bim):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_bim_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_test)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_bim_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_test)

In [28]:
if (loading_bim):
    print("loading...")
    numpy_raw_train_bim = np.load("D:/data_adversarial_july/"+dataset+"/images_bim_train.npy")
    labels_to_adv_onehot_train_bim = np.load("D:/data_adversarial_july/"+dataset+"/labels_bim_train.npy")
    
    numpy_raw_test_bim = np.load("D:/data_adversarial_july/"+dataset+"/images_bim_test.npy")
    labels_to_adv_onehot_test_bim = np.load("D:/data_adversarial_july/"+dataset+"/labels_bim_test.npy")
    print("finished")

loading...
finished


In [29]:
model.evaluate(numpy_raw_test_bim,labels_to_adv_onehot_test_bim)

32/32 [==============================] - 1s 40ms/step - loss: 0.5234 - accuracy: 0.8090


[0.5233824849128723, 0.8090000152587891]

### FGSM

In [73]:
images_to_adv_train,labels_to_adv_train  = get_from_original_set_adv(idxs_fgsm_train,x_train,y_train_labels)

5082
--- 0
label 2.0
2224
--- 1
label 1.0
8390
--- 2
label 4.0
361
--- 3
label 0.0
9668
--- 4
label 5.0
8495
--- 5
label 4.0
6043
--- 6
label 3.0
4075
--- 7
label 2.0
1986
--- 8
label 1.0
4447
--- 9
label 2.0
5748
--- 10
label 3.0
5336
--- 11
label 2.0
1710
--- 12
label 0.0
7278
--- 13
label 3.0
5896
--- 14
label 3.0
226
--- 15
label 0.0
1471
--- 16
label 0.0
837
--- 17
label 0.0
5696
--- 18
label 3.0
3289
--- 19
label 1.0
9777
--- 20
label 5.0
5023
--- 21
label 2.0
1259
--- 22
label 0.0
7773
--- 23
label 4.0
3604
--- 24
label 2.0
10739
--- 25
label 5.0
1729
--- 26
label 0.0
7610
--- 27
label 4.0
3172
--- 28
label 1.0
7405
--- 29
label 3.0
8190
--- 30
label 4.0
6239
--- 31
label 3.0
9405
--- 32
label 5.0
7932
--- 33
label 4.0
5570
--- 34
label 3.0
5334
--- 35
label 2.0
9595
--- 36
label 5.0
10410
--- 37
label 5.0
137
--- 38
label 0.0
11122
--- 39
label 5.0
8640
--- 40
label 4.0
8754
--- 41
label 4.0
8281
--- 42
label 4.0
10093
--- 43
label 5.0
8675
--- 44
label 4.0
3808
--- 45
label 2.

--- 455
label 5.0
3071
--- 456
label 1.0
6270
--- 457
label 3.0
2063
--- 458
label 1.0
435
--- 459
label 0.0
6581
--- 460
label 3.0
5470
--- 461
label 2.0
10584
--- 462
label 5.0
1107
--- 463
label 0.0
6505
--- 464
label 3.0
8935
--- 465
label 4.0
8282
--- 466
label 4.0
11080
--- 467
label 5.0
2129
--- 468
label 1.0
8559
--- 469
label 4.0
2030
--- 470
label 1.0
937
--- 471
label 0.0
9307
--- 472
label 4.0
1084
--- 473
label 0.0
4666
--- 474
label 2.0
4784
--- 475
label 2.0
3786
--- 476
label 2.0
688
--- 477
label 0.0
1510
--- 478
label 0.0
5331
--- 479
label 2.0
6997
--- 480
label 3.0
2311
--- 481
label 1.0
1883
--- 482
label 1.0
2492
--- 483
label 1.0
4298
--- 484
label 2.0
9964
--- 485
label 5.0
8687
--- 486
label 4.0
9849
--- 487
label 5.0
7822
--- 488
label 4.0
11062
--- 489
label 5.0
4694
--- 490
label 2.0
1745
--- 491
label 0.0
6658
--- 492
label 3.0
6348
--- 493
label 3.0
1312
--- 494
label 0.0
11152
--- 495
label 5.0
158
--- 496
label 0.0
8480
--- 497
label 4.0
3668
--- 498
lab

--- 852
label 3.0
5635
--- 853
label 3.0
9380
--- 854
label 5.0
2488
--- 855
label 1.0
1660
--- 856
label 0.0
3843
--- 857
label 2.0
150
--- 858
label 0.0
6069
--- 859
label 3.0
1500
--- 860
label 0.0
9876
--- 861
label 5.0
7579
--- 862
label 4.0
10982
--- 863
label 5.0
9886
--- 864
label 5.0
2829
--- 865
label 1.0
8818
--- 866
label 4.0
8398
--- 867
label 4.0
8615
--- 868
label 4.0
6331
--- 869
label 3.0
885
--- 870
label 0.0
8810
--- 871
label 4.0
1611
--- 872
label 0.0
5522
--- 873
label 3.0
9771
--- 874
label 5.0
1256
--- 875
label 0.0
10329
--- 876
label 5.0
8135
--- 877
label 4.0
2344
--- 878
label 1.0
4419
--- 879
label 2.0
11056
--- 880
label 5.0
6302
--- 881
label 3.0
7585
--- 882
label 4.0
1646
--- 883
label 0.0
5928
--- 884
label 3.0
1769
--- 885
label 1.0
1951
--- 886
label 1.0
8430
--- 887
label 4.0
6648
--- 888
label 3.0
4459
--- 889
label 2.0
5340
--- 890
label 2.0
1530
--- 891
label 0.0
857
--- 892
label 0.0
9203
--- 893
label 4.0
9584
--- 894
label 5.0
5924
--- 895
lab

In [74]:
images_to_adv_test,labels_to_adv_test  = get_from_original_set_adv(idxs_fgsm_test,x_test,y_test_labels)

625
--- 0
label 1.0
1194
--- 1
label 2.0
2315
--- 2
label 4.0
2287
--- 3
label 4.0
624
--- 4
label 1.0
766
--- 5
label 1.0
2359
--- 6
label 4.0
2465
--- 7
label 4.0
2825
--- 8
label 5.0
2991
--- 9
label 5.0
866
--- 10
label 1.0
1244
--- 11
label 2.0
2172
--- 12
label 4.0
2540
--- 13
label 5.0
1918
--- 14
label 3.0
117
--- 15
label 0.0
936
--- 16
label 2.0
910
--- 17
label 1.0
1423
--- 18
label 2.0
63
--- 19
label 0.0
2493
--- 20
label 4.0
2854
--- 21
label 5.0
2187
--- 22
label 4.0
2736
--- 23
label 5.0
2370
--- 24
label 4.0
1388
--- 25
label 2.0
1631
--- 26
label 3.0
1824
--- 27
label 3.0
1172
--- 28
label 2.0
655
--- 29
label 1.0
2441
--- 30
label 4.0
2716
--- 31
label 5.0
1656
--- 32
label 3.0
558
--- 33
label 1.0
2671
--- 34
label 5.0
2905
--- 35
label 5.0
957
--- 36
label 2.0
748
--- 37
label 1.0
167
--- 38
label 0.0
2809
--- 39
label 5.0
1891
--- 40
label 3.0
2331
--- 41
label 4.0
679
--- 42
label 1.0
1744
--- 43
label 3.0
2998
--- 44
label 5.0
1245
--- 45
label 2.0
691
--- 46
la

label 1.0
2534
--- 416
label 5.0
1583
--- 417
label 3.0
1562
--- 418
label 3.0
683
--- 419
label 1.0
1618
--- 420
label 3.0
2115
--- 421
label 4.0
1430
--- 422
label 2.0
2872
--- 423
label 5.0
1513
--- 424
label 3.0
1441
--- 425
label 2.0
1488
--- 426
label 3.0
2224
--- 427
label 4.0
2690
--- 428
label 5.0
1677
--- 429
label 3.0
797
--- 430
label 1.0
2390
--- 431
label 4.0
2079
--- 432
label 4.0
2766
--- 433
label 5.0
2926
--- 434
label 5.0
801
--- 435
label 1.0
436
--- 436
label 0.0
2336
--- 437
label 4.0
858
--- 438
label 1.0
500
--- 439
label 1.0
294
--- 440
label 0.0
2742
--- 441
label 5.0
2820
--- 442
label 5.0
133
--- 443
label 0.0
579
--- 444
label 1.0
1228
--- 445
label 2.0
2177
--- 446
label 4.0
1694
--- 447
label 3.0
2007
--- 448
label 4.0
2670
--- 449
label 5.0
738
--- 450
label 1.0
1580
--- 451
label 3.0
2068
--- 452
label 4.0
2126
--- 453
label 4.0
1785
--- 454
label 3.0
1023
--- 455
label 2.0
2185
--- 456
label 4.0
1321
--- 457
label 2.0
141
--- 458
label 0.0
342
--- 459


893
--- 952
label 1.0
2294
--- 953
label 4.0
1955
--- 954
label 3.0
1462
--- 955
label 2.0
622
--- 956
label 1.0
994
--- 957
label 2.0
2715
--- 958
label 5.0
376
--- 959
label 0.0
2202
--- 960
label 4.0
802
--- 961
label 1.0
1440
--- 962
label 2.0
2232
--- 963
label 4.0
1948
--- 964
label 3.0
2277
--- 965
label 4.0
2445
--- 966
label 4.0
1075
--- 967
label 2.0
400
--- 968
label 0.0
2300
--- 969
label 4.0
1217
--- 970
label 2.0
1505
--- 971
label 3.0
2022
--- 972
label 4.0
1696
--- 973
label 3.0
563
--- 974
label 1.0
128
--- 975
label 0.0
2323
--- 976
label 4.0
2213
--- 977
label 4.0
25
--- 978
label 0.0
907
--- 979
label 1.0
2446
--- 980
label 4.0
2598
--- 981
label 5.0
350
--- 982
label 0.0
1702
--- 983
label 3.0
1052
--- 984
label 2.0
1447
--- 985
label 2.0
1712
--- 986
label 3.0
2828
--- 987
label 5.0
346
--- 988
label 0.0
2240
--- 989
label 4.0
2463
--- 990
label 4.0
1464
--- 991
label 3.0
1964
--- 992
label 3.0
1027
--- 993
label 2.0
861
--- 994
label 1.0
966
--- 995
label 2.0
165

In [75]:
labels_to_adv_train=tf.convert_to_tensor(labels_to_adv_train)
images_to_adv_train=tf.convert_to_tensor(images_to_adv_train)
images_to_adv_train = tf.cast(images_to_adv_train, tf.float32)
labels_to_adv_train = tf.cast(labels_to_adv_train, tf.int32)

In [76]:
labels_to_adv_test = tf.convert_to_tensor(labels_to_adv_test)
images_to_adv_test = tf.convert_to_tensor(images_to_adv_test)
images_to_adv_test = tf.cast(images_to_adv_test, tf.float32)
labels_to_adv_test = tf.cast(labels_to_adv_test, tf.int32)

In [77]:
attack_FGSM = fb.attacks.FGSM()

In [78]:
start_time = datetime.now()

raw_train, clipped, is_adv = attack_FGSM(fmodel, images_to_adv_train, labels_to_adv_train,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:05.473828


In [79]:
start_time = datetime.now()

raw_test, clipped, is_adv = attack_FGSM(fmodel, images_to_adv_test, labels_to_adv_test,epsilons=0.05)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:06.382652


In [80]:
numpy_raw_train = K.eval(raw_train)
numpy_labels_train = K.eval(labels_to_adv_train)

In [81]:
numpy_raw_test = K.eval(raw_test)
numpy_labels_test = K.eval(labels_to_adv_test)

In [82]:
labels_to_adv_onehot_train =np_utils.to_categorical(labels_to_adv_train, model.num_classes)
labels_to_adv_onehot_test =np_utils.to_categorical(labels_to_adv_test, model.num_classes)

In [30]:
if (loading_fgsm):
    print("loading...")
    numpy_raw_train_fgsm = np.load("D:/data_adversarial_july/"+dataset+"/images_fgsm_train.npy")
    labels_to_adv_onehot_train_fgsm = np.load("D:/data_adversarial_july/"+dataset+"/labels_fgsm_train.npy")
    
    numpy_raw_test_fgsm = np.load("D:/data_adversarial_july/"+dataset+"/images_fgsm_test.npy")
    labels_to_adv_onehot_test_fgsm = np.load("D:/data_adversarial_july/"+dataset+"/labels_fgsm_test.npy")
    print("finished")

loading...
finished


In [84]:
if(not loading_fgsm):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_fgsm_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_train)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_fgsm_train.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_train)

In [85]:
if (not loading_fgsm):
    save_dir = "D:/data_adversarial_july/"+dataset+"/images_fgsm_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, numpy_raw_test)

    save_dir = "D:/data_adversarial_july/"+dataset+"/labels_fgsm_test.npy"
    with open(save_dir, 'wb') as f:
        np.save(f, labels_to_adv_onehot_test)

In [31]:
model.evaluate(numpy_raw_test_fgsm,labels_to_adv_onehot_test_fgsm)

32/32 [==============================] - 1s 39ms/step - loss: 3.1869 - accuracy: 0.1490


[3.186912775039673, 0.14900000393390656]

In [87]:
pwd

'C:\\Users\\fjdur\\Documents\\utils'

In [32]:
print(x_train.shape)
print(numpy_raw_train.shape)
x_train_and_adversary = np.concatenate((x_train,numpy_raw_train),axis=0)
print(x_train_and_adversary.shape)
x_train_and_adversary = np.concatenate((x_train_and_adversary,numpy_raw_train_bim),axis=0)
print(x_train_and_adversary.shape)
x_train_and_adversary = np.concatenate((x_train_and_adversary,numpy_raw_train_fgsm),axis=0)
print(x_train_and_adversary.shape)

print(y_train.shape)
print(labels_to_adv_onehot_train.shape)
y_train_and_adversary= np.concatenate((y_train,labels_to_adv_onehot_train),axis=0)
print(y_train_and_adversary.shape)
y_train_and_adversary= np.concatenate((y_train_and_adversary,labels_to_adv_onehot_train_bim),axis=0)
print(y_train_and_adversary.shape)
y_train_and_adversary= np.concatenate((y_train_and_adversary,labels_to_adv_onehot_train_fgsm),axis=0)
print(y_train_and_adversary.shape)


(11224, 48, 48, 1)
(1000, 48, 48, 1)
(12224, 48, 48, 1)
(13224, 48, 48, 1)
(14224, 48, 48, 1)
(11224, 6)
(1000, 6)
(12224, 6)
(13224, 6)
(14224, 6)


In [33]:
print(x_test.shape)
print(numpy_raw_test.shape)
x_test_and_adversary = np.concatenate((x_test,numpy_raw_test),axis=0)
print(x_test_and_adversary.shape)
x_test_and_adversary = np.concatenate((x_test_and_adversary,numpy_raw_test_bim),axis=0)
print(x_test_and_adversary.shape)
x_test_and_adversary = np.concatenate((x_test_and_adversary,numpy_raw_test_fgsm),axis=0)
print(x_test_and_adversary.shape)
#labels_to_adv_onehot_test

print(y_test.shape)
#print(labels_test_to_adv_oneshot.shape)
y_test_and_adversary= np.concatenate((y_test,labels_to_adv_onehot_test),axis=0)
print(y_test_and_adversary.shape)
y_test_and_adversary= np.concatenate((y_test_and_adversary,labels_to_adv_onehot_test_bim),axis=0)
print(y_test_and_adversary.shape)
y_test_and_adversary= np.concatenate((y_test_and_adversary,labels_to_adv_onehot_test_fgsm),axis=0)
print(y_test_and_adversary.shape)


(3000, 48, 48, 1)
(1000, 48, 48, 1)
(4000, 48, 48, 1)
(5000, 48, 48, 1)
(6000, 48, 48, 1)
(3000, 6)
(4000, 6)
(5000, 6)
(6000, 6)


In [34]:
model.evaluate(x_test_and_adversary,y_test_and_adversary)

188/188 [==============================] - 7s 37ms/step - loss: 0.9981 - accuracy: 0.6872


[0.9981359839439392, 0.687166690826416]

In [ ]:
save_dir = "D:/data_adversarial_july/"+dataset+"/train_and_adversary.npy"
with open(save_dir, 'wb') as f:
    np.save(f, x_train_and_adversary)

save_dir = "D:/data_adversarial_july/"+dataset+"/train_and_adversary_labels.npy"    
with open(save_dir, 'wb') as f:
    np.save(f, y_train_and_adversary)

In [ ]:
save_dir = "D:/data_adversarial_july/"+dataset+"/test_and_adversary.npy"
with open(save_dir, 'wb') as f:
    np.save(f, x_test_and_adversary)

save_dir = "D:/data_adversarial_july/"+dataset+"/test_and_adversary_labels.npy"
with open(save_dir, 'wb') as f:
    np.save(f, y_test_and_adversary)